In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from pathlib import Path

In [2]:
download_dir = Path("../data").resolve()

In [3]:
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : str(download_dir)}
#chrome_options.add_argument("--headless")
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.geolocation": 2})
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
service = ChromeService(executable_path=ChromeDriverManager().install())

C:\Users\ebroh\AppData\Local\Temp\ipykernel_2476\1938636411.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
C:\Users\ebroh\AppData\Local\Temp\ipykernel_2476\1938636411.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


In [4]:
df1 = pd.read_csv('https://data.cityofnewyork.us/api/views/hebw-6hze/rows.csv?accessType=DOWNLOAD')
df1['Zip Code'] = df1['Zip Code'].astype(str)
zip_codes = df1['Zip Code'].unique()

In [5]:
#counties = ['Bronx', 'Brooklyn', 'Queens', 'Manhattan', 'Staten Island']
counties = ['Bronx']

clinics = []

for zip_code in counties:
#for zip_code in zip_codes:
    driver.get('https://findaddictiontreatment.ny.gov/')
    
    xpath = '//*[@id="oasasbed"]/div[1]/div[2]/div/div[2]/form/div[5]/div/input'
    
    try:
        # Wait until the element is clickable
        searchBar = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
        
    except:
        continue

    searchBar.clear()
    searchBar.send_keys(zip_code)
    searchBar.send_keys(Keys.ENTER)
    
    driver.find_element(By.XPATH,'//*[@id="oasasbed"]/div[1]/div[2]/div/div[2]/form/div[2]/div/div/div[2]/label/input').click()
    driver.find_element(By.XPATH,'//*[@id="oasasbed"]/div[1]/div[2]/div/div[2]/form/div[8]/div/div/button[1]').click()
    
    xpath = '//*[@id="oasasbed"]/div[1]/div[2]/div/div[2]/div/div/div[2]/button[4]'

    try:
        # Wait until the element is clickable
        element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
    
    except:
        continue

    # Click the element
    element.click()
    
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    cards = soup.find_all(attrs={"ng-attr-id": "card-{{item.properties.id1}}"})
    for card in cards:
        facility_name = card.find('h3', class_='panel-title ng-binding').text.strip()
        miles = card.find_all('h3', class_='panel-title ng-binding')[1].text.strip()
        address_raw = soup.find('a', class_='ng-binding').contents
        address = ' '.join([str(x).strip() for x in address_raw if not str(x).startswith('<')])
        phone_number = card.find('a', {'href': lambda x: x and x.startswith('tel:')}).text.strip()

        availability_table = card.find('table', class_='table text-center')
        adult_availability = ''
        adolescent_availability = ''
        female_availability = ''
        male_availability = ''
        transgender_availability = ''

        if availability_table:
            for tr in availability_table.find_all('tr', class_='ng-scope'):
                td = tr.find('td')
                if td:
                    td_text = td.text.strip()
                    availability = bool(tr.find('i', class_='fa fa-check ng-scope'))

                    if td_text == 'Adult':
                        adult_availability = availability
                    elif td_text == 'Adolescent':
                        adolescent_availability = availability
                    elif td_text == 'Female':
                        female_availability = availability
                    elif td_text == 'Male':
                        male_availability = availability
                    elif td_text == 'Transgender':
                        transgender_availability = availability

        next_availability = ""
        next_availability_element = card.find('h4', text='Next Availability')
        if next_availability_element:
            next_availability = next_availability_element.next_sibling.strip()

        adult_beds = card.find('h4', class_='ng-binding')
        adult_beds = adult_beds.text.split(':')[1].strip() if adult_beds and 'Adult Beds' in adult_beds.text else ''

        clinics.append({
            'Facility Name': facility_name,
            'Miles': miles,
            'Address': address,
            'Phone Number': phone_number,
            'Adult Availability': adult_availability,
            'Adolescent Availability': adolescent_availability,
            'Female Availability': female_availability,
            'Male Availability': male_availability,
            'Transgender Availability': transgender_availability,
            'Next Availability': next_availability,
            'Adult Beds': adult_beds
        })

In [6]:
clinics

[{'Facility Name': 'Counseling Services of New York LLC OP Outpatient Clinic',
  'Miles': '0.13 Miles',
  'Address': '911 Walton Ave Bronx, NY 10452-9554',
  'Phone Number': '718-590-1790',
  'Adult Availability': True,
  'Adolescent Availability': False,
  'Female Availability': '',
  'Male Availability': '',
  'Transgender Availability': '',
  'Next Availability': '(Please Call for Appointment)',
  'Adult Beds': ''},
 {'Facility Name': 'Next Steps South OP Outpatient Clinic',
  'Miles': '0.26 Miles',
  'Address': '911 Walton Ave Bronx, NY 10452-9554',
  'Phone Number': '718-993-3397',
  'Adult Availability': True,
  'Adolescent Availability': False,
  'Female Availability': '',
  'Male Availability': '',
  'Transgender Availability': '',
  'Next Availability': '(Please Call for Appointment)',
  'Adult Beds': ''},
 {'Facility Name': 'Wellness Center at Melrose OTP Opioid Outpatient Treatment',
  'Miles': '0.26 Miles',
  'Address': '911 Walton Ave Bronx, NY 10452-9554',
  'Phone Number

In [7]:
clinics_df = pd.DataFrame(clinics)
clinics_df

,Facility Name,Miles,Address,Phone Number,Adult Availability,Adolescent Availability,Female Availability,Male Availability,Transgender Availability,Next Availability,Adult Beds
0,Counseling Services of New York LLC OP Outpati...,0.13 Miles,"911 Walton Ave Bronx, NY 10452-9554",718-590-1790,True,False,,,,(Please Call for Appointment),
1,Next Steps South OP Outpatient Clinic,0.26 Miles,"911 Walton Ave Bronx, NY 10452-9554",718-993-3397,True,False,,,,(Please Call for Appointment),
2,Wellness Center at Melrose OTP Opioid Outpatie...,0.26 Miles,"911 Walton Ave Bronx, NY 10452-9554",718-993-3397,,,,,,(Please Call for Appointment),
3,"Concourse Medical Center, Inc. OTP Opioid Outp...",0.3 Miles,"911 Walton Ave Bronx, NY 10452-9554",718-665-9340,,,,,,(Please Call for Appointment),
4,"Argus Community, Inc. RR 1 820 Rehab and Reint...",0.58 Miles,"911 Walton Ave Bronx, NY 10452-9554",718-993-2376,,,True,True,False,04/13/2023,2
...,...,...,...,...,...,...,...,...,...,...,...
84,"Inwood Community Services, Inc. OP Outpatient ...",2.76 Miles,"911 Walton Ave Bronx, NY 10452-9554",212-942-0043,True,True,,,,(Please Call for Appointment),
85,Veritas House RRi 820 Residential Reintegration,2.77 Miles,"911 Walton Ave Bronx, NY 10452-9554",646-790-7364,,,True,True,True,04/14/2023,9
86,Samaritan Daytop Village Inc. IR 4 Intensive R...,2.77 Miles,"911 Walton Ave Bronx, NY 10452-9554",212-222-5150,,,True,False,False,04/14/2023,16
87,The Center for Wellness and Change OP Outpatie...,2.79 Miles,"911 Walton Ave Bronx, NY 10452-9554",212-663-3000,True,False,,,,(Please Call for Appointment),


In [8]:
clinics_df.to_csv(str(download_dir / Path('OASAS_full.csv')))